In [1]:
import os
import sys
import scipy.stats

import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression 
from sklearn.metrics import classification_report  
from sklearn.linear_model import LogisticRegression
from scipy.signal import butter, filtfilt
from scipy.signal import detrend

plt.style.use('ggplot')

In [4]:
base_path = os.getcwd()
base_path = base_path.replace('notebooks', 'scripts') # note: if you have notebooks twice in your base_path, this won't work
sys.path.insert(1, base_path)

from helpers import *
from utils import remove_top_right_frame, jitter_scatterplot
from function import *

In [5]:
main_dir =  r"C:\Users\user\Desktop\EPFL\MA1\projet_cellular\neuro_circuit"
#main_dir = r"C:/Users/maxca/Documents/EPFL/Computational_neuro/BIO482_MiniProject_2023_Python"
print('Main working directory:', main_dir)
figure_path = os.path.join(main_dir, 'Figures')
os.makedirs(figure_path, exist_ok=True)
data_path = os.path.join(main_dir, 'Data') #if your folder is organized differently, just specify the full path to the .mat file

fname = 'data_bio482-002.plk'
data_df = pd.read_pickle(os.path.join(data_path, fname))

Main working directory: C:\Users\user\Desktop\EPFL\MA1\projet_cellular\neuro_circuit


In [ ]:
# Select the relevant columns from data_df and assign them to cells_df
cells_df = data_df[['Cell_ID', 'Cell_Type', 'Cell_APThreshold_Slope', 'Cell_Depth', 'Cell_Layer', 'Cell_TargetedBrainArea', 'Sweep_MembranePotential']]
sampling_rate = 20000

In [ ]:
cell = 17
noisy_cell = 5

# Extracting data for a specific cell from the DataFrame
Vm = cells_df['Sweep_MembranePotential'].iloc[cell]  # Get membrane potential for the cell at index 'cell'
#deriv_thresh = cells_df['Cell_APThreshold_Slope'].iloc[cell]  # Get action potential threshold slope for the same cell

# Extracting similar data for another cell, referred to as 'noisy_cell'
Vm2 = cells_df['Sweep_MembranePotential'].iloc[noisy_cell]  # Get membrane potential for the cell at index 'noisy_cell'
#deriv_thresh2 = cells_df['Cell_APThreshold_Slope'].iloc[noisy_cell]  # Get action potential threshold slope for 'noisy_cell'

In [ ]:
lower_threshold = lower_bound(Vm, mvg_avg_window = 1000, sub_window_size=20000)
upper_threshold = 0.004

spikelets = find_spikelet_moving_avg(Vm, window_size=window_size, lower_threshold=lower_threshold, SR_Vm=sampling_rate)
spikelet_testplot(Vm, spikelets)

spikelets2 = find_spikelet_moving_avg(Vm2, window_size=window_size, lower_threshold=lower_threshold, SR_Vm=sampling_rate)
spikelet_testplot(Vm2, spikelets2)